## IMDB and Amazon Review: Text Classification with SpaCy

3 Datasets. In each dataset, 0=negative sentiment, 1=positive sentiment

In [48]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

## Import, Clean, Explore Data

In [84]:
yelp_dF = pd.read_csv('../Review_Classification/yelp_labelled.txt', sep='\t')
amazon_dF = pd.read_csv('../Review_Classification/amazon_cells_labelled.txt', sep='\t')
imdb_dF = pd.read_csv('../Review_Classification/imdb_labelled.txt', sep='\t')
##0-negative, 1-positive

In [85]:
print(f'{yelp_dF.head()}\n{amazon_dF.head()}\n{imdb_dF.head()}')

                            Wow... Loved this place.  1
0                                 Crust is not good.  0
1          Not tasty and the texture was just nasty.  0
2  Stopped by during the late May bank holiday of...  1
3  The selection on the menu was great and so wer...  1
4     Now I am getting angry and I want my damn pho.  0
  So there is no way for me to plug it in here in the US unless I go by a converter.  \
0                        Good case, Excellent value.                                   
1                             Great for the jawbone.                                   
2  Tied to charger for conversations lasting more...                                   
3                                  The mic is great.                                   
4  I have to jiggle the plug to get it to line up...                                   

   0  
0  1  
1  1  
2  0  
3  1  
4  0  
  A very, very, very slow-moving, aimless movie about a distressed, drifting young man.    \


In [86]:
## Name Columns
dataframes = [amazon_dF, imdb_dF, yelp_dF]
column_names = ['Review', 'Sentiment']
for dF in dataframes:
    dF.columns = column_names

In [87]:
amazon_dF.head()

,Review,Sentiment
0,"Good case, Excellent value.",1
1,Great for the jawbone.,1
2,Tied to charger for conversations lasting more...,0
3,The mic is great.,1
4,I have to jiggle the plug to get it to line up...,0


In [88]:
yelp_dF.head()

,Review,Sentiment
0,Crust is not good.,0
1,Not tasty and the texture was just nasty.,0
2,Stopped by during the late May bank holiday of...,1
3,The selection on the menu was great and so wer...,1
4,Now I am getting angry and I want my damn pho.,0


In [89]:
imdb_dF.head()

,Review,Sentiment
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0


In [90]:
amazon_dF.shape, imdb_dF.shape, yelp_dF.shape

((999, 2), (747, 2), (999, 2))

In [91]:
amazon_dF.isna().sum(), imdb_dF.isna().sum(), yelp_dF.isna().sum()

(Review       0
 Sentiment    0
 dtype: int64,
 Review       0
 Sentiment    0
 dtype: int64,
 Review       0
 Sentiment    0
 dtype: int64)

No missing values to worry about. Data is clean. No create one super dF.

In [92]:
reviews_dF = yelp_dF.append([amazon_dF, imdb_dF], ignore_index=True)
reviews_dF.shape

(2745, 2)

In [93]:
reviews_dF['Sentiment'].value_counts()

1    1385
0    1360
Name: Sentiment, dtype: int64

Balanced Dataset. Now apply some SpaCy techniques to remove stop words & punctuation and lemmatize.

## SpaCy: Stop Words & Lemmatization

In [94]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
punct = string.punctuation

In [95]:
stop_words = list(STOP_WORDS)
stop_words

['side',
 'before',
 'sometime',
 'neither',
 'both',
 'formerly',
 'became',
 'hereby',
 'is',
 'just',
 'no',
 'less',
 'the',
 'those',
 'was',
 'or',
 'without',
 'yourselves',
 'did',
 'every',
 'here',
 'other',
 'ten',
 'another',
 'his',
 'per',
 'yours',
 'ca',
 'about',
 'below',
 'how',
 'at',
 'itself',
 'might',
 'first',
 'thereby',
 'top',
 'also',
 'could',
 'using',
 'namely',
 'we',
 'bottom',
 'us',
 'again',
 'off',
 'herself',
 'several',
 'such',
 'whose',
 'back',
 'nine',
 'nowhere',
 'their',
 'and',
 'three',
 'above',
 'not',
 'who',
 'latter',
 'only',
 'these',
 'except',
 'whoever',
 'else',
 'ourselves',
 'within',
 'quite',
 'unless',
 'into',
 'myself',
 'make',
 'whether',
 'indeed',
 'her',
 'should',
 'than',
 'so',
 'down',
 'hence',
 'be',
 'name',
 're',
 'as',
 'made',
 'yourself',
 'fifty',
 'put',
 'still',
 'themselves',
 'everything',
 'least',
 'me',
 'mostly',
 'none',
 'something',
 'otherwise',
 'whatever',
 'either',
 'does',
 'already',

In [96]:
nlp = spacy.load('en_core_web_sm')

def text_data_cleaner(sentence):
    doc = nlp(sentence)
    tokens = []
    
    for token in doc:
        if token.lemma_ != '-PRON-':
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
        
    cleaned_tokens = []
    for token in tokens:
        if token not in stop_words and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

If root form of word (lemma) is not a pronoun, then we convert into lower form; if proper noun, then we directly take lower form because there's no lemma 

In [97]:
text_data_cleaner('Hello, I am studying machine learning and NLP. I am a programmer.')

['hello', 'study', 'machine', 'learning', 'nlp', 'programmer']

## Vectorization & Feature Engineering
Using scikit-learn's text frequency-inverse document frequency vectorizer and linear support vector classifier.

In [98]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [99]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaner)
clf = LinearSVC()

## Train Model

In [100]:
from sklearn.model_selection import train_test_split
X = reviews_dF['Review']
y = reviews_dF['Sentiment']


In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=16)

In [102]:
X_train.shape, X_test.shape

((1839,), (906,))

In [104]:
clf_model = Pipeline([('tfidf', tfidf), ('clf', clf)])
clf_model.fit(X_train, y_train)

/Users/finneassensiba/anaconda3/envs/nlp_course/lib/python3.7/site-packages/sklearn/utils/validation.py:209: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(joblib_version) < '0.12':


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

## Make Predictions

In [105]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
predictions = clf_model.predict(X_test)

In [108]:
print(f'{classification_report(y_test, predictions)}\nAccuracy: {math.ceil(100*(accuracy_score(y_test, predictions)))}%\nConfusion Matrix:\n{confusion_matrix(y_test, predictions)}')

              precision    recall  f1-score   support

           0       0.81      0.77      0.79       460
           1       0.77      0.81      0.79       446

   micro avg       0.79      0.79      0.79       906
   macro avg       0.79      0.79      0.79       906
weighted avg       0.79      0.79      0.79       906

Accuracy: 79%
Confusion Matrix:
[[354 106]
 [ 85 361]]
